In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video

In [ ]:
#Helper functions
def display_two_images(img1,img2):
    _,axes = plt.subplots(1,2,figsize=(10,5))
    axes[0].imshow(img1)
    axes[1].imshow(img2)

In [ ]:
# Read and show an image
def read_and_show_image(fn):
    image = cv2.imread(fn)
    window = cv2.namedWindow('Image',cv2.WINDOW_AUTOSIZE)
    cv2.imshow(window,image)
    cv2.waitKey(0)
    cv2.destroyWindow('Image')

#read_and_show_image('images/tower_bridge.jpg')

In [ ]:
# Read and show a video
def read_and_show_video(fn):
    window = cv2.namedWindow('Video',cv2.WINDOW_AUTOSIZE)
    cap = cv2.VideoCapture()
    if cap.open(fn):
        (success,frame) = cap.read()
        while(success):
            cv2.imshow(window,frame)
            if(cv2.waitKey(33) >=0):
                break
            (success,frame) = cap.read()
    cv2.destroyWindow('Video')

#read_and_show_video('videos/piano.mp4')

In [ ]:
# Add a trackbar slider to the video viewer for moving around 
# within the video file

In [ ]:
# Read and show image inline
def read_and_convert_image(fn):
    image = cv2.imread(fn)
    return cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

plt.imshow(read_and_convert_image('images/tower_bridge.jpg'));


In [ ]:
# smooth image
def smooth_image(image):
    image = cv2.GaussianBlur(image,(5,5),3)
    image = cv2.GaussianBlur(image,(5,5),3)
    return image

image = read_and_convert_image('images/tower_bridge.jpg')
out = smooth_image(image)
display_two_images(image,out)

Plain **downsampling** introduces high frequencies into the resulting image. So run a high pass filter and then downsample. Do this using the **pyrDown** method.**pyramid down**

In [ ]:
image = read_and_convert_image("images/tower_bridge.jpg")
display_two_images(image,cv2.pyrDown(image,image))

**Canny edge detector** writes to a single channel grayscale image. So convert the image first to grayscale and then run through the edge detector. 

In [ ]:
def cannyEdgeDetect(image):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    edge_image = np.zeros_like(image)
    edge_image = None
    return cv2.Canny(image,threshold1=10,
                     threshold2=100,apertureSize=3,L2gradient=True)

image = read_and_convert_image("images/tower_bridge.jpg")
edge = cannyEdgeDetect(image)
_,axes = plt.subplots(1,2,figsize=(10,5))
axes[0].imshow(image)
axes[1].imshow(edge,cmap='gray')
plt.show()

Using pyrDown and canny, get a cleaner primary edge detection.

In [ ]:
def primaryEdgeDetect(image):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    image = cv2.pyrDown(image)
    image = cv2.pyrDown(image)
    image = cv2.Canny(image,threshold1=10,threshold2=100,
                     apertureSize=3,L2gradient=True)
    return image
image = read_and_convert_image("images/tower_bridge.jpg")
edge = primaryEdgeDetect(image)
_,axes = plt.subplots(1,2,figsize=(10,5))
axes[0].imshow(image)
axes[1].imshow(edge,cmap='gray')
plt.show()

**Log polar transformed** format is like what our eye would see. Convert a video to this format and save.

In [ ]:
def convertVideoToLogPolarTransform(ipfn,opfn):
    cap = cv2.VideoCapture(ipfn)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    writer = cv2.VideoWriter(opfn,cv2.VideoWriter_fourcc('M','J','P','G'),
                           fps,(width,height))
    (success,frame) = cap.read()
    while success:
        lpframe = cv2.logPolar(frame,(frame.shape[0]/2,frame.shape[1]/2),
                    40,cv2.WARP_FILL_OUTLIERS)
        writer.write(lpframe)
        (success,frame) = cap.read()
    cap.release()
    writer.release()
    return True
retval = convertVideoToLogPolarTransform('videos/piano.mp4',
                                         'videos/piano_lpf.mp4')
print(retval)

In [ ]:
Video("videos/piano_lpf.mp4",width=500,height=500)